In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install rich[jupyter]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/summary/data/dataset_train.csv",encoding='utf-8')
df=pd.DataFrame((df),columns=["original","summary"])
df

,original,summary
0,"Ngày 28/7 , Trung_tâm báo tin động_đất và cảnh...","Trong vòng ba ngày , lưu_vực thuỷ_điện Sông_Tr..."
1,Hãng tin Reuters ngày 6-5 dẫn lời người_phát_n...,Quân_đội Mỹ cho_biết ngày 6-5 hai tàu_khu_trục...
2,Khu trục hạm INS Kolkata của hải_quân Ấn_Độ . ...,Ấn_Độ và Singapore triển_khai nhiều khí_tài th...
3,"Chiều 6-10 , đại_tá Nguyễn_Trọng_Hà - trưởng_p...",Bốn cán_bộ tại Ngân_hàng Agribank chi_nhánh hu...
4,Đây là cơn mưa_đá thứ hai xảy ra trong tháng 4...,Chỉ quét qua 15 phút nhưng những viên đá khiến...
...,...,...
7995,Video : Công_an khám nhà ông Nguyễn_Ngọc_Tuấn ...,Ông Tuấn bị khởi_tố vì những sai_phạm lúc còn ...
7996,"Theo tiết_lộ của một quan_chức Nhà_Trắng , con...",Việc con_gái ông Trump trở_thành nhân_viên của...
7997,Lính Mỹ huấn_luyện tấn_công hệ_thống hầm ngầm ...,Bình_Nhưỡng khẳng_định việc đào_tạo lính chiến...
7998,"Theo BBC , lực_lượng cứu_hộ cần ít_nhất 10 giờ...",Sau khi đưa được 4 người đầu_tiên trong nhóm 1...


In [3]:
df

,original,summary
0,"Ngày 28/7 , Trung_tâm báo tin động_đất và cảnh...","Trong vòng ba ngày , lưu_vực thuỷ_điện Sông_Tr..."
1,Hãng tin Reuters ngày 6-5 dẫn lời người_phát_n...,Quân_đội Mỹ cho_biết ngày 6-5 hai tàu_khu_trục...
2,Khu trục hạm INS Kolkata của hải_quân Ấn_Độ . ...,Ấn_Độ và Singapore triển_khai nhiều khí_tài th...
3,"Chiều 6-10 , đại_tá Nguyễn_Trọng_Hà - trưởng_p...",Bốn cán_bộ tại Ngân_hàng Agribank chi_nhánh hu...
4,Đây là cơn mưa_đá thứ hai xảy ra trong tháng 4...,Chỉ quét qua 15 phút nhưng những viên đá khiến...
...,...,...
7995,Video : Công_an khám nhà ông Nguyễn_Ngọc_Tuấn ...,Ông Tuấn bị khởi_tố vì những sai_phạm lúc còn ...
7996,"Theo tiết_lộ của một quan_chức Nhà_Trắng , con...",Việc con_gái ông Trump trở_thành nhân_viên của...
7997,Lính Mỹ huấn_luyện tấn_công hệ_thống hầm ngầm ...,Bình_Nhưỡng khẳng_định việc đào_tạo lính chiến...
7998,"Theo BBC , lực_lượng cứu_hộ cần ít_nhất 10 giờ...",Sau khi đưa được 4 người đầu_tiên trong nhóm 1...


In [5]:
import os
import numpy as np
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,AutoTokenizer
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("orginal", justify="center" ), Column("summary", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)

In [6]:
if torch.cuda.is_available():       
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [8]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [9]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [10]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="/content/drive/MyDrive/summary/vi_model" ):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True


  # tokenzier for encoding the text
  tokenizer = AutoTokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)



  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text,target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text,target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
      

  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
 
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))

In [11]:
model_params={
    "MODEL":"NlpHUST/t5-small-vi-summarization",            
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":5,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":64,   # max length of target text
    "SEED": 42                     # set seed for reproducibility 

}

In [12]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [ ]:
T5Trainer(dataframe=df, source_text="original", target_text="summary", model_params=model_params, output_dir="/content/drive/MyDrive/summary/vi_model")

Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/572 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


...

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(2.9746, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(2.5566, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(2.3069, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(2.7733, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(2.2196, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(2.2571, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  60   | tensor(1.9100, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  70   | tensor(2.0402, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  80   | tensor(2.4126, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  90   | tensor(2.1233, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  100  | tensor(2.1625, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  110  | tensor(2.3785, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  120  | tensor(1.8578, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  130  | tensor(2.4200, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  140  | tensor(2.1877, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  150  | tensor(2.0592, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  160  | tensor(2.0561, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  170  | tensor(1.6697, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  180  | tensor(2.4419, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  190  | tensor(1.7736, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  200  | tensor(2.0405, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  210  | tensor(2.3393, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  220  | tensor(1.7728, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  230  | tensor(2.1142, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  240  | tensor(1.6230, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  250  | tensor(2.0645, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  260  | tensor(1.7763, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  270  | tensor(1.8589, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  280  | tensor(1.6088, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  290  | tensor(2.2094, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  300  | tensor(1.8867, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  310  | tensor(1.8887, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  320  | tensor(1.9564, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  330  | tensor(1.7623, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  340  | tensor(1.6808, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  350  | tensor(1.7998, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  360  | tensor(1.6462, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  370  | tensor(1.8984, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  380  | tensor(1.7947, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  390  | tensor(1.6710, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  400  | tensor(1.6463, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  410  | tensor(1.9233, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  420  | tensor(1.5977, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  430  | tensor(1.8549, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  440  | tensor(1.6102, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  450  | tensor(1.4817, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  460  | tensor(2.0410, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  470  | tenso

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(2.9746, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(2.5566, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(2.3069, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(2.7733, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(2.2196, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(2.2571, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  60   | tensor(1.9100, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  70   | tensor(2.0402, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  80   | tensor(2.4126, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  90   | tensor(2.1233, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  100  | tensor(2.1625, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  110  | tensor(2.3785, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  120  | tensor(1.8578, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  130  | tensor(2.4200, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  140  | tensor(2.1877, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  150  | tensor(2.0592, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  160  | tensor(2.0561, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  170  | tensor(1.6697, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  180  | tensor(2.4419, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  190  | tensor(1.7736, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  200  | tensor(2.0405, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  210  | tensor(2.3393, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  220  | tensor(1.7728, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  230  | tensor(2.1142, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  240  | tensor(1.6230, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  250  | tensor(2.0645, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  260  | tensor(1.7763, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  270  | tensor(1.8589, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  280  | tensor(1.6088, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  290  | tensor(2.2094, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  300  | tensor(1.8867, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  310  | tensor(1.8887, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  320  | tensor(1.9564, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  330  | tensor(1.7623, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  340  | tensor(1.6808, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  350  | tensor(1.7998, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  360  | tensor(1.6462, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  370  | tensor(1.8984, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  380  | tensor(1.7947, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  390  | tensor(1.6710, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  400  | tensor(1.6463, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  410  | tensor(1.9233, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  420  | tensor(1.5977, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  430  | tensor(1.8549, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  440  | tensor(1.6102, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  450  | tensor(1.4817, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  460  | tensor(2.0410, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  470  | tenso

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(2.9746, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(2.5566, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(2.3069, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(2.7733, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(2.2196, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(2.2571, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  60   | tensor(1.9100, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  70   | tensor(2.0402, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  80   | tensor(2.4126, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  90   | tensor(2.1233, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  100  | tensor(2.1625, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  110  | tensor(2.3785, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  120  | tensor(1.8578, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  130  | tensor(2.4200, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  140  | tensor(2.1877, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  150  | tensor(2.0592, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  160  | tensor(2.0561, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  170  | tensor(1.6697, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  180  | tensor(2.4419, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  190  | tensor(1.7736, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  200  | tensor(2.0405, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  210  | tensor(2.3393, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  220  | tensor(1.7728, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  230  | tensor(2.1142, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  240  | tensor(1.6230, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  250  | tensor(2.0645, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  260  | tensor(1.7763, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  270  | tensor(1.8589, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  280  | tensor(1.6088, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  290  | tensor(2.2094, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  300  | tensor(1.8867, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  310  | tensor(1.8887, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  320  | tensor(1.9564, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  330  | tensor(1.7623, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  340  | tensor(1.6808, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  350  | tensor(1.7998, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  360  | tensor(1.6462, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  370  | tensor(1.8984, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  380  | tensor(1.7947, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  390  | tensor(1.6710, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  400  | tensor(1.6463, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  410  | tensor(1.9233, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  420  | tensor(1.5977, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  430  | tensor(1.8549, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  440  | tensor(1.6102, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  450  | tensor(1.4817, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  460  | tensor(2.0410, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  470  | tenso

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(2.9746, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(2.5566, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(2.3069, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(2.7733, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(2.2196, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(2.2571, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  60   | tensor(1.9100, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  70   | tensor(2.0402, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  80   | tensor(2.4126, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  90   | tensor(2.1233, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  100  | tensor(2.1625, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  110  | tensor(2.3785, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  120  | tensor(1.8578, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  130  | tensor(2.4200, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  140  | tensor(2.1877, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  150  | tensor(2.0592, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  160  | tensor(2.0561, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  170  | tensor(1.6697, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  180  | tensor(2.4419, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  190  | tensor(1.7736, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  200  | tensor(2.0405, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  210  | tensor(2.3393, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  220  | tensor(1.7728, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  230  | tensor(2.1142, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  240  | tensor(1.6230, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  250  | tensor(2.0645, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  260  | tensor(1.7763, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  270  | tensor(1.8589, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  280  | tensor(1.6088, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  290  | tensor(2.2094, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  300  | tensor(1.8867, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  310  | tensor(1.8887, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  320  | tensor(1.9564, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  330  | tensor(1.7623, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  340  | tensor(1.6808, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  350  | tensor(1.7998, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  360  | tensor(1.6462, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  370  | tensor(1.8984, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  380  | tensor(1.7947, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  390  | tensor(1.6710, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  400  | tensor(1.6463, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  410  | tensor(1.9233, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  420  | tensor(1.5977, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  430  | tensor(1.8549, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  440  | tensor(1.6102, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  450  | tensor(1.4817, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  460  | tensor(2.0410, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  470  | tenso

                               Training Status                               
+---------------------------------------------------------------------------+
|Epoch | Steps |                            Loss                            |
|------+-------+------------------------------------------------------------|
|  0   |   0   | tensor(2.9746, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(2.5566, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(2.3069, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(2.7733, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(2.2196, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(2.2571, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  60   | tensor(1.9100, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  70   | tensor(2.0402, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  80   | tensor(2.4126, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  90   | tensor(2.1233, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  100  | tensor(2.1625, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  110  | tensor(2.3785, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  120  | tensor(1.8578, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  130  | tensor(2.4200, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  140  | tensor(2.1877, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  150  | tensor(2.0592, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  160  | tensor(2.0561, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  170  | tensor(1.6697, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  180  | tensor(2.4419, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  190  | tensor(1.7736, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  200  | tensor(2.0405, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  210  | tensor(2.3393, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  220  | tensor(1.7728, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  230  | tensor(2.1142, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  240  | tensor(1.6230, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  250  | tensor(2.0645, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  260  | tensor(1.7763, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  270  | tensor(1.8589, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  280  | tensor(1.6088, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  290  | tensor(2.2094, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  300  | tensor(1.8867, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  310  | tensor(1.8887, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  320  | tensor(1.9564, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  330  | tensor(1.7623, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  340  | tensor(1.6808, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  350  | tensor(1.7998, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  360  | tensor(1.6462, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  370  | tensor(1.8984, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  380  | tensor(1.7947, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  390  | tensor(1.6710, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  400  | tensor(1.6463, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  410  | tensor(1.9233, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  420  | tensor(1.5977, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  430  | tensor(1.8549, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  440  | tensor(1.6102, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  450  | tensor(1.4817, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  460  | tensor(2.0410, device='cuda:0', grad_fn=<NllLossBackward0>)|
|  0   |  470  | tenso

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/summary/vi_model/model_files")
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/summary/vi_model/model_files")

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

text = '''Chúng mình thường chỉ biết đến một Hà Nội với tên gọi là "chốn phồn hoa đô hội", một Hà Nội xa hoa, tráng lệ, đông đúc và xô bồ. 
Nhưng có ai từng thắc mắc rằng, Hà Nội của những thập niên về trước trông thế nào, Hà Nội trong kí ức của những người đã gắn bó với Hà Nội hơn nửa đời người trông thế nào không?
“Góc phố Hà Trung-Ngõ Trạm có bốt điện chợ Hàng Da,nơi gắn kỷ niệm tuổi thơ tôi thường trèo bàng lấy lá bán cho Cá cảnh để họ bán giun.Thích thì đổi cá chọi về nuôi trong bóng tối chút ,sau mang ra "chọi' với hàng xóm vui lắm vì cá chọi nuôi trong bóng tối lúc ra oánh nhau ác liệt lắm và thường là thắng.”
“Thủa nhỏ sợ nhất là được giao nhiệm vụ đi đổi bún ở chợ, vừa đông, vừa mùi chua chua của gạo ngâm. Hàng cá cảnh luôn có vài ba bạn quần đùi áo may ô đứng chọn cá chọi, bạn gái xem cá chỉ đứng xa xa thôi. Nhưng đó là trước năm 80, còn sau này thì…đã lớn tôi không còn được nhìn thấy những hình ảnh đó nữa.”
“Được nhìn thấy cảnh này cảm xúc lại trào dâng , những ký ức lại hiện ra trong đầu của hơn 50 năm về trước cũng chỗ này nhưng trải qua nhiều biến đổi theo từng khoảng thời gian. Có lúc là điểm bán cá cảnh và cũng có lúc là chỗ tập trung của chợ hoa quả như trong hình. Chỗ bán cá cảnh trước 75 , ngày xưa cứ đi tàu điện đến cửa nhà em là xuống đi bộ qua Ngõ Trạm đến đây mua cá Chọi về giữa trưa hè đầy tiếng Ve kêu.”
“Tháng năm tuổi thơ như vừa mới hôm qua, tự dưng lại nhớ cái mùi hung húc hàng khô với cá thịt tanh tanh, xếp hàng tem phiếu mua mỡ.... Nhớ nhất chỗ đó bán rau, cạnh đó bán thịt. Mỗi lần xe rau về là lao ra xếp hàng. Không nhanh họ bán hết mà không mua được thì chiều về cách cụ lại o.ánh cho to.ét mông.”
“Vâng, trước “cái món khoái khẩu của tôi” là cá chọi (chọi cá…). Nhà thì ở đấy nhưng cũng đã có nhiều lần còn lên tận cả Yên Phụ, Nghi Tàm để mua cá …! Còn nữa, những trường Thăng Long, Điện Biên, TQToản, Tân Trào, Hoàn Kiếm…, những kí ức của tuổi thơ ngày nào nay lại chợt ập về.”
Đôi khi người ta yêu một thành phố, không phải chỉ vì vẻ đẹp riêng biệt của nó, mà còn vì những điều xưa cũ, vì ở nơi đó từng chất chứa cả một bầu trời tuổi thơ..'''

print("original text: \n",text)
preprocess_text = text.strip().replace("\n","")
tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)

summary_ids = model.generate(
            tokenized_text,
            max_length=256, 
            num_beams=2,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True
        )

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized: \n",output)

original text: 
 Chúng mình thường chỉ biết đến một Hà Nội với tên gọi là "chốn phồn hoa đô hội", một Hà Nội xa hoa, tráng lệ, đông đúc và xô bồ. 
Nhưng có ai từng thắc mắc rằng, Hà Nội của những thập niên về trước trông thế nào, Hà Nội trong kí ức của những người đã gắn bó với Hà Nội hơn nửa đời người trông thế nào không?
“Góc phố Hà Trung-Ngõ Trạm có bốt điện chợ Hàng Da,nơi gắn kỷ niệm tuổi thơ tôi thường trèo bàng lấy lá bán cho Cá cảnh để họ bán giun.Thích thì đổi cá chọi về nuôi trong bóng tối chút ,sau mang ra "chọi' với hàng xóm vui lắm vì cá chọi nuôi trong bóng tối lúc ra oánh nhau ác liệt lắm và thường là thắng.”
“Thủa nhỏ sợ nhất là được giao nhiệm vụ đi đổi bún ở chợ, vừa đông, vừa mùi chua chua của gạo ngâm. Hàng cá cảnh luôn có vài ba bạn quần đùi áo may ô đứng chọn cá chọi, bạn gái xem cá chỉ đứng xa xa thôi. Nhưng đó là trước năm 80, còn sau này thì…đã lớn tôi không còn được nhìn thấy những hình ảnh đó nữa.”
“Được nhìn thấy cảnh này cảm xúc lại trào dâng , những ký ức 

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
tokenizer.tokenize('i love you')

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


['▁', 'i', '▁love', '▁you']